In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: Evaluating batch prediction results from a custom tabular classification model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/custom_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/custom_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/custom_tabular_classification_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook demonstrates how to use the Vertex AI classification model evaluation component to evaluate a custom-trained tabular classification model saved in Vertex AI Model Registry. Model evaluation helps you determine your model performance based on the evaluation metrics and improve the model if necessary. 

### Objective

In this tutorial, you train a scikit-learn RandomForest model, save it in Vertex AI Model Registry and learn how to evaluate it through a Vertex AI pipeline job using `google_cloud_pipeline_components`.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model Registry
- Vertex AI Pipelines
- Vertex AI Batch Predictions

The steps performed include:

- Fetch the dataset from the public source.
- Preprocess the data locally and save test data in BigQuery.
- Train a RandomForest classification model locally using scikit-learn Python package.
- Create a custom container in Artifact Registry for predictions.
- Upload the model in Vertex AI Model Registry.
- Create and run a Vertex AI Pipeline that:
    - Imports the trained model into the pipeline.
    - Runs a `Batch Prediction` job on the test data in BigQuery.
    - Evaulates the model using the evaluation component from google-cloud-pipeline-components Python SDK.
    - Imports the classification metrics in to the model resource in Vertex AI Model Registry.
- Print and visualize the classification evaluation metrics.
- Clean up the resources created in this notebook.

### Dataset

The **Census Income Dataset** used in this notebook, is hosted on BigQuery under public datasets. Originally, it is sourced by the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php). The underlying task associated with the dataset is to determine whether a person makes over 50K a year or not. For more information, check the [details on its UCI webpage](https://archive.ics.uci.edu/ml/datasets/Census+Income).

### Costs 
This tutorial uses billable components of Google Cloud:

* Artifact Registry
* BigQuery
* Cloud Build
* Cloud Storage
* Vertex AI

Learn about [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing), [Cloud Build pricing](https://cloud.google.com/build/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installation

Install the following packages required to execute this notebook. 

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the latest versions of the following packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-pipeline-components \
                        matplotlib pyarrow {USER_FLAG} -q
# Install the specified versions of the following packages
! pip3 install scikit-learn==1.0 \
                pandas \
                joblib==1.2.0 \
                numpy==1.23.3 {USER_FLAG} -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, Compute Engine, Artifact Registry, Cloud Build and Dataflow APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute.googleapis.com,artifactregistry.googleapis.com,cloudbuild.googleapis.com,dataflow.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. It is recommended that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click **Create**. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you run a Vertex AI pipeline job using the Cloud SDK, your job stores the pipeline artifacts in a Cloud Storage bucket. In this tutorial, you create a Vertex AI Pipeline job that saves the artifacts like evaluation metrics and feature attributes to a Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

#### Service Account

You use a service account to create Vertex AI Pipeline jobs. If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Import libraries

Import the Vertex AI Python SDK and other required Python libraries.

In [ ]:
import os

import joblib
import kfp
import matplotlib.pyplot as plt
from google.cloud import aiplatform, aiplatform_v1, bigquery
from kfp.v2 import compiler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import LabelBinarizer

### Initialize Vertex AI and BigQuery SDK for Python

Initialize the Vertex AI and BigQuery SDK for Python with your project and the created bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

### Define constants

In the next cell, define the constants that you are going to use in this session.

In [ ]:
# Define the public bigquery data source
DATA_SOURCE = "bigquery-public-data.ml_datasets.census_adult_income"
# Define the table name for storing the test data
PREDICTION_INPUT_DATASET_ID = f"income_prediction_{UUID}"
# Set the folder path inside GCS bucket where you store model artifacts
MODEL_ARTIFACT_DIR = "custom-sklearn-income-pred-model"
# Set the name of the local folder where you store your prediction application
SRC_DIR = "src"
# Define the feature columns that you use from the dataset
COLUMNS = (
    "age",
    "workclass",
    "functional_weight",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
)
# Categorical columns are columns that have string values and
# need to be turned into a numerical value to be used for training
CATEGORICAL_COLUMNS = (
    "workclass",
    "education",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native_country",
)
# Target column in the dataset
TARGET = "income_bracket"
# Save the individual class labels in a constant
CLASS_LABELS = [" <=50K", " >50K"]
# Set the test ratio for splitting
TEST_SIZE = 0.25
# Set a random state
RANDOM_STATE = 36
# Set a sample size for batch prediction
BATCH_SAMPLE_SIZE = 3000
# Set the name for your repository in artifact registry
REPOSITORY = "sklearn-inc-pred-custom-cont-repo"
# Set the name for your prediction image in artifact registry
IMAGE = "sklearn-fastapi-server"
# Set a display name for your Vertex AI Model
MODEL_DISPLAY_NAME = "skl_inc_pred_model_" + UUID
# Set a display name for your Vertex AI Pipeline
PIPELINE_DISPLAY_NAME = "income_classification_multiclass_" + UUID
# Path where the compiled pipeline needs to be written
PIPELINE_PACKAGE_NAME = f"custom_tabular_classify_pipeline_{UUID}.json"
# Set the GCS path to your root directory for Vertex AI pipelines
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/income_classification_task_{UUID}"

## Fetch the dataset

Download the census data from the BigQuery public dataset. In this tutorial, you only use a few rows(20K) from the dataset for training and testing.

In [ ]:
# Define the SQL query to fetch the dataset
query = f"""
SELECT * FROM `{DATA_SOURCE}` LIMIT 20000
"""
# Download the dataset to a dataframe
df = bq_client.query(query).to_dataframe()
df.head()

## Split the data

Divide the data into train and test. You train the Random Forest classification model on the train set and use the test data for evaluation. 

In [ ]:
# Split the dataset
X_train, X_test = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_STATE)
# Print the shapes of train and test sets
print(X_train.shape, X_test.shape)

## Save the test data in BigQuery

Create a dataset in BigQuery and store the test set in a table inside the dataset. This dataset is further used while running the evaluation pipeline for creating data samples and storing predictions.

### Create a dataset in BigQuery

In [ ]:
# Create a bigquery dataset
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

### Configure the schema for storing test data

In [ ]:
schema_config = []
for i in COLUMNS:
    if X_test[i].dtype == "int64":
        schema_config.append(bigquery.SchemaField(i, "INTEGER"))
    elif X_test[i].dtype in ["object", "category"]:
        schema_config.append(bigquery.SchemaField(i, "STRING"))

schema_config.append(bigquery.SchemaField(TARGET, "STRING"))

### Load the test data to a table

In [ ]:
table_ref = bq_dataset.table("adult_income_test_data")
job_config = bigquery.LoadJobConfig(
    schema=schema_config, write_disposition="WRITE_TRUNCATE"
)

job = bq_client.load_table_from_dataframe(X_test, table_ref)

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

## Create a preprocessing pipeline for training

Since the dataset consists of both categorical and numerical data, certain steps of preprocessing are required. However, your dataset needs to have only numerical values before it can be used for training the classification model. Therefore, you encode the categorical data to numerical values using [LabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) as given below.

To keep it simple while serving predictions, the following code encapsulates the steps in a scikit-learn Pipeline. You can export Pipeline objects using the version of `joblib` included in scikit-learn or `pickle`, similarly to how you export [scikit-learn estimators](https://scikit-learn.org/stable/tutorial/statistical_inference/settings.html#estimators-objects).

Learn more about [scikit-learn Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [ ]:
# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
train_features = X_train.drop(TARGET, axis=1).to_numpy().tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
train_labels = X_train[TARGET].to_numpy().tolist()

# Since the census data set has categorical features, we need to convert
# them to numerical values. We'll use a list of pipelines to convert each
# categorical column and then use FeatureUnion to combine them before calling
# the RandomForestClassifier.
categorical_pipelines = []

# Each categorical column needs to be extracted individually and converted to a numerical value.
# To do this, each categorical column will use a pipeline that extracts one feature column via
# SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
# A scores array (created below) will select and extract the feature column. The scores array is
# created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
for i, col in enumerate(COLUMNS):
    if col in CATEGORICAL_COLUMNS:
        # Create a scores array to get the individual categorical column.
        # Example:
        #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical',
        #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
        #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #
        # Returns: [['Sate-gov']]
        scores = []
        # Build the scores array
        for j in range(len(COLUMNS)):
            if i == j:  # This column is the categorical column we want to extract.
                scores.append(1)  # Set to 1 to select this column
            else:  # Every other column should be ignored.
                scores.append(0)
        skb = SelectKBest(k=1)
        skb.scores_ = scores
        # Convert the categorical column to a numerical value
        lbn = LabelBinarizer()
        r = skb.transform(train_features)
        lbn.fit(r)
        # Create the pipeline to extract the categorical feature
        categorical_pipelines.append(
            (
                "categorical-{}".format(i),
                Pipeline([("SKB-{}".format(i), skb), ("LBN-{}".format(i), lbn)]),
            )
        )

# Create pipeline to extract the numerical features
skb = SelectKBest(k=6)
# From COLUMNS use the features that are numerical
skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
categorical_pipelines.append(("numerical", skb))

# Combine all the features using FeatureUnion
preprocess = FeatureUnion(categorical_pipelines)

## Train a Random Forest classification model

Next, you fit a Random Forest classification model on the preprocessed data.

After training, you add the estimator to the pipeline object and save the pipeline to disk.

In [ ]:
# Create the classifier
classifier = RandomForestClassifier()

# Transform the features and fit them to the classifier
classifier.fit(preprocess.transform(train_features), train_labels)

# Create the overall model as a single pipeline
pipeline = Pipeline([("union", preprocess), ("classifier", classifier)])

# Save the pipeline
joblib.dump(pipeline, "model.joblib")

## Create a container image for serving predictions

For serving predictions using the model, you upload the model to Vertex AI Model Registry using a custom container for predictions.
To create the container image, you take the following steps:
- Save the model to your Cloud Storage bucket.
- Locally, create an application using for serving using [`FastAPI`](https://fastapi.tiangolo.com/tutorial/first-steps/) Python package.
- Dockerize the application and upload it to [Artifact Registry using Cloud Build](https://cloud.google.com/build/docs/build-push-docker-image).

Learn more about using a [custom container for predictions on Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/use-custom-container).

### Upload the model to Cloud Storage bucket

In [ ]:
!gsutil cp model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

### Create the serving application 

Create a source directory where you pacakge your serving application.

In [ ]:
# Create the source directory
! mkdir $SRC_DIR
# Create the app folder
! mkdir $SRC_DIR/app

Create the `main.py` file that contains the code to serve predictions using `FastAPI`.

In [ ]:
%%writefile $SRC_DIR/app/main.py
# Import the required libraries
from fastapi import FastAPI, Request
import joblib
import json
import os
from google.cloud import storage
import logging

# Define the Cloud Storage client
gcs_client = storage.Client()

# Download the model file from Cloud Storage bucket
with open("model.joblib", 'wb') as model_f:
    gcs_client.download_blob_to_file(
            f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
        )
    
# Load the scikit-learn model/pipeline file
_model = joblib.load("model.joblib")

# Define a function for health route
@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

# Define a function for prediction route
@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    # await the request
    body = await request.json()
    # parse the request instances
    instances = body["instances"]
    # pass it to the model/pipeline for prediction scores
    predictions = _model.predict_proba(instances).tolist()
    # return the batch prediction scores
    return {"predictions": predictions}

# main
if __name__=="__main__":
    app = FastAPI()

Create `requirements.txt` file that specifies the package versions for the application.

In [ ]:
%%writefile $SRC_DIR/requirements.txt
joblib==1.2.0
numpy==1.23.3
scikit-learn==1.0
google-cloud-storage>=1.44.0,<2.0.0dev

Create the `prestart.sh` bash script that sets the port to `AIP_HTTP_PORT`. Your container's HTTP server listens for requests on this port.

In [ ]:
%%writefile $SRC_DIR/app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

### Containerize the serving application

Create a Dockerfile for containerizing the serving application.

In [ ]:
%%writefile $SRC_DIR/Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

#### Create a repository

To store your container image, create a repository in the Artifact Registry.

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

#### Push the container image
Using Cloud Build, containerize your serving application and push it to your repository.

In [ ]:
%cd $SRC_DIR/
!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}
%cd ..

## Upload the model to Vertex AI Registry

Now, create a Vertex AI model using the container image and the artifact directory path in Cloud Storage bucket where your model was uploaded. 

To upload your model, you use the `Model.upload()` method from Vertex AI SDK by passing the following parameters:

- `display_name`: Display name of the model resource.
- `artifact_uri`: Cloud Storage path where the model file/artifact(s) is located.
- `serving_container_image_uri`: Path to the serving container image.
- `serving_container_predict_route`: Serving application's predict route.
- `serving_container_health_route`: Serving application's health check route.

In [ ]:
aip_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    serving_container_predict_route="/predict",
    serving_container_health_route="/health",
)

## Create and run the evaluation pipeline

In this section, you run a Vertex AI Pipeline that runs the following steps:
1. Imports your model from Vertex AI Model Registry.
1. Samples the test data for batch prediction.
1. Removes the target field from the sampled test data.
1. Runs the batch prediction job.
1. Evaluates results from the batch prediction job using the ground-truth/target information.
1. Imports the generated evaluation metrics to the Vertex AI model.


### Define the pipeline
To define the Vertex AI Pipeline for evaluating your model, you use the `google-cloud-pipeline-components` Python package. Google Cloud Pipeline Components provides an SDK with a set of pipeline components for users to interact with Google Cloud services such as Vertex AI, Dataflow and BigQuery. 

Learn more about [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction).

The evaluation pipeline uses the following components:

- `GetVertexModelOp`: Gets a Vertex AI Model artifact. 
- `EvaluationDataSamplerOp`: Randomly downsamples an input dataset to a specified size for computing Vertex Explainable AI feature attributions for AutoML Tabular and custom models. Creates a Dataflow job with Apache Beam to downsample the dataset. 
- `TargetFieldDataRemoverOp`: Removes the target field from the input dataset for supporting unstructured AutoML models and custom models for Vertex AI batch prediction.
- `ModelBatchPredictOp`: Creates a Vertex AI batch prediction job and waits for it to complete. 
- `ModelEvaluationClassificationOp`: Compute evaluation metrics on a trained model’s batch prediction results. Creates a Dataflow job with Apache Beam and TFMA to compute evaluation metrics. Supports mutliclass classification evaluation for tabular, image, video, and text data. 
- `ModelImportEvaluationOp`: Imports a model evaluation artifact to an existing Vertex AI model with ModelService.ImportModelEvaluation. 

In [ ]:
# define the evaluation pipeline
@kfp.dsl.pipeline(name="custom-tabular-classification-evaluation-pipeline")
def evaluation_custom_tabular_feature_attribution_pipeline(
    project: str,
    location: str,
    root_dir: str,
    model_name: str,
    target_column_name: str,
    bigquery_source_input_uri: str,
    bigquery_destination_output_uri: str,
    batch_predict_instances_format: str,
    evaluation_class_names: list,
    batch_predict_predictions_format: str = "bigquery",
    evaluation_prediction_label_column: str = "",
    evaluation_prediction_score_column: str = "prediction",
    batch_predict_machine_type: str = "n1-standard-4",
    batch_predict_starting_replica_count: int = 5,
    batch_predict_max_replica_count: int = 10,
    batch_predict_data_sample_size: int = 10000,
):
    # Import the components
    from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
    from google_cloud_pipeline_components.experimental.evaluation import (
        EvaluationDataSamplerOp, GetVertexModelOp,
        ModelEvaluationClassificationOp, ModelImportEvaluationOp,
        TargetFieldDataRemoverOp)

    # Get the Vertex AI model resource
    get_model_task = GetVertexModelOp(model_resource_name=model_name)

    # Run the data sampling task
    data_sampler_task = EvaluationDataSamplerOp(
        project=project,
        location=location,
        root_dir=root_dir,
        bigquery_source_uri=bigquery_source_input_uri,
        instances_format=batch_predict_instances_format,
        sample_size=batch_predict_data_sample_size,
    )

    # Run the task to remove the target field from data for batch prediction
    data_splitter_task = TargetFieldDataRemoverOp(
        project=project,
        location=location,
        root_dir=root_dir,
        bigquery_source_uri=data_sampler_task.outputs["bigquery_output_table"],
        instances_format=batch_predict_instances_format,
        target_field_name=target_column_name,
    )

    # Run the batch prediction task
    batch_predict_task = ModelBatchPredictOp(
        project=project,
        location=location,
        model=get_model_task.outputs["model"],
        job_display_name="model-registry-batch-prediction",
        bigquery_source_input_uri=data_splitter_task.outputs["bigquery_output_table"],
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predict_predictions_format,
        bigquery_destination_output_uri=bigquery_destination_output_uri,
        machine_type=batch_predict_machine_type,
        starting_replica_count=batch_predict_starting_replica_count,
        max_replica_count=batch_predict_max_replica_count,
    )

    # Run the evaluation based on prediction type
    eval_task = ModelEvaluationClassificationOp(
        project=project,
        location=location,
        root_dir=root_dir,
        class_labels=evaluation_class_names,
        prediction_label_column=evaluation_prediction_label_column,
        prediction_score_column=evaluation_prediction_score_column,
        target_field_name=target_column_name,
        ground_truth_format=batch_predict_instances_format,
        ground_truth_bigquery_source=data_sampler_task.outputs["bigquery_output_table"],
        predictions_format=batch_predict_predictions_format,
        predictions_bigquery_source=batch_predict_task.outputs["bigquery_output_table"],
    )

    # Import the model evaluations to the Vertex AI model
    ModelImportEvaluationOp(
        classification_metrics=eval_task.outputs["evaluation_metrics"],
        model=get_model_task.outputs["model"],
        dataset_type=batch_predict_instances_format,
    )

### Compile the evaluation pipeline

Compile the defined pipeline to a (json/yaml) file.

In [ ]:
compiler.Compiler().compile(
    pipeline_func=evaluation_custom_tabular_feature_attribution_pipeline,
    package_path=PIPELINE_PACKAGE_PATH,
)

### Define the parameters

Before running your pipeline, set the following parameters :

- `project`: Project ID of the Google Cloud project.
- `location`: Region where the pipeline needs to be run. If not set, the pipeline defaults to the region that Vertex AI SDK is configured with.
- `root_dir`: The Cloud Storage directory for keeping the staged files and artifacts. A random subdirectory is created under the directory to keep the job information for resuming the job in case of a failure.
- `model_name`: Resource name of the trained custom tabular classification model.
- `target_column_name`: Name of the column to be used as the ground truth for evaluation.
- `bigquery_source_input_uri`: BigQuery table URI where the test input is stored.
- `bigquery_destination_output_uri`: BigQuery dataset URI for exporting predictions on the test set.
- `batch_predict_instances_format`: Format of the input for batch prediction and evaluation.
- `batch_predict_predictions_format`: Format of the output for batch prediction and evaluation.
- `evaluation_class_names`: The list of all class names for the target field in the dataset. 
- `batch_predict_data_sample_size`: Sample size of the input test data needed for batch prediction job and evaluation.

In [ ]:
parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "root_dir": PIPELINE_ROOT,
    "model_name": aip_model.resource_name,
    "target_column_name": TARGET,
    "bigquery_source_input_uri": f"bq://{PROJECT_ID}.{table_ref.dataset_id}.{table_ref.table_id}",
    "bigquery_destination_output_uri": f"bq://{PROJECT_ID}.{table_ref.dataset_id}",
    "batch_predict_instances_format": "bigquery",
    "batch_predict_predictions_format": "bigquery",
    "evaluation_class_names": CLASS_LABELS,
    "batch_predict_data_sample_size": BATCH_SAMPLE_SIZE,
}

### Run the pipeline

Create a Vertex AI pipeline job using the following parameters and run it:

- `display_name`: The name of the pipeline that should show up in the Google Cloud console.
- `template_path`: The path of compiled PipelineSpec JSON or YAML file. It can be a local path, a Google Cloud Storage URI or an Artifact Registry URI.
- `parameter_values`: The mapping from runtime parameter names to its values that control the pipeline run.
- `enable_caching`: Boolean to spcify whether to turn on caching for the run or not.

Learn more about Vertex AI SDK's [PipelineJob Class](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob).

After creating the pipeline job, run it using the configured `SERVICE_ACCOUNT`.

In [ ]:
# Create the pipeline job
job = aiplatform.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path=PIPELINE_PACKAGE_PATH,
    parameter_values=parameters,
    enable_caching=True,
)
# Run the pipeline job
job.run(service_account=SERVICE_ACCOUNT)

## Print the metrics

After the pipeline has run successfully, fetch the evaluation metrics from the evaluation task and print them.

In [ ]:
# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[0]
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

## Visualize the metrics

Visualize the generated evaluation metrics using a bar chart.

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(5, 3))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

- Vertex AI Model
- Vertex AI Pipeline job
- Repository in Artifact Registry
- BigQuery dataset
- Cloud Storage bucekt (set `delete_bucket` to **True** to create the Cloud Storage bucket created in this notebook).

In [ ]:
delete_bucket = False

# Delete model resource
model.delete()

# Delete the evaluation pipeline
job.delete()

# Delete the repository in Artifact Registry
! gcloud artifacts repositories delete --location=us-central1 {REPOSITORY} --quiet

# Delete the BigQuery dataset
bq_dataset.delete(force=True)

# Delete Cloud Storage objects
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI